In [11]:
'''
In the messy dataset the following things have to be cleaned up: 
- missing values 
- random string values
- swapped columns 
- numeric distortion (numbers have been multiplied, scaled or randomly altered) 
- truncated text
'''
import pandas as pd
import numpy as np
import math

In [ ]:
# there are over 79 colums a lot of which contains filler information
# i wanted to look at this from a business perspective so i recorded my insights and figured out what columns could be useful for this
# the rest of the columms, since they are not relevant have been dropped
df = pd.read_csv("../../data/raw/detailed_listings_data.csv")

import pandas as pd
import numpy as np
import re

# Filters out any unnecessary columns
def filter_columns(df):
    col_for_insights = [
        'id', 'property_type', 'room_type',
        'price', 'estimated_revenue_l365d',
        'accommodates', 'beds', 'bedrooms', 'bathrooms',
        'review_scores_rating', 'number_of_reviews', 'reviews_per_month',
        'availability_365', 'host_response_rate', 'host_response_time', 'host_since',
        'amenities', 'minimum_nights', 'maximum_nights',
        'latitude', 'longitude', 'neighbourhood_cleansed',
        'host_total_listings_count', 'host_is_superhost',
        'review_scores_cleanliness', 'review_scores_value', 'host_acceptance_rate'
    ]

    filtered_df = df[col_for_insights]
    
    return filtered_df

# Converts object dtype columns to string dtype
def convert_objects_to_string(df):
    obj_cols = df.select_dtypes(include="object").columns
    df[obj_cols] = df[obj_cols].astype("string")

    return df

# Converts price column to integer
def convert_price_to_int(df):
    if "price" in df.columns:
        df["price"] = (
            df["price"]
            .astype("string")
            .str.replace(r"[^0-9]", "", regex=True)   # remove $, commas
            .replace("", np.nan).astype("float")
            .astype("Int64")
        )

    return df

# Converts beds and bedrooms into integer
def convert_bed_columns_to_int(df):
    for col in ["beds", "bedrooms"]:
        if col in df.columns:
            df[col] = (
                df[col].astype("string").replace("", np.nan)
                .astype("float").astype("Int64")
            )

    return df

# Converts host response & acceptance rate to integer from string percentage "97%"
def convert_rates_to_int(df):
    for col in ["host_response_rate", "host_acceptance_rate"]:
        if col in df.columns:
            df[col] = (
                df[col].astype("string")
                .str.replace("%", "", regex=False)
                .replace("", np.nan).astype("float").astype("Int64")
            )

    return df

# Converts host_since column into datetime
def convert_host_since(df):
    if "host_since" in df.columns:
        df["host_since"] = pd.to_datetime(df["host_since"], errors="coerce")

    return df

# Checks each indiividual value to see if it has t or f (or any other way of specifying)
def convert_superhost_to_bool(df):
    if "host_is_superhost" in df.columns:
        df["host_is_superhost"] = df["host_is_superhost"].apply(
            lambda x: True if str(x).strip().lower() in ["t", "true", "yes", "y"]
            else False if str(x).strip().lower() in ["f", "false", "no", "n"]
            else pd.NA
        ).astype("boolean")

    return df
    
#    This drops rows where the percentage of missing values exceeds the given threshold.    
#    the default threshold is 0.40 means rows with more  than 40% missing values are removed.
def drop_rows_with_missing_threshold(df, threshold=0.40):
    
    # Calculates fraction of missing values per row
    missing_fraction = df.isna().mean(axis=1)
    
    # Identifies rows to drop
    rows_to_drop = missing_fraction[missing_fraction > threshold].index
    
    # Drops them
    df_cleaned = df.drop(index=rows_to_drop)
    
    return df_cleaned

# Cleans the amenities column by removing brackets/quotes and converting into a list of strings.
def clean_amenities_column(df):
    
    df["amenities"] = df["amenities"].apply( # Applies a transformation to every value 
        lambda value: [] if pd.isna(value) # If empty just return an empty list
        else [
            item.strip() # Filters out any empty strings
            for item in (value.replace("[", "").replace("]", "")
                        .replace('"', "").replace("'", "")
            ).split(",") # splits the strings into separate list items by the comma
            if item.strip()
        ]
    )
    
    return df


def add_price_competitiveness(df):
    # Measures how competitively priced a listing is relative to its neighborhood & property type.
    
    group_median = df.groupby(
        ["neighbourhood_cleansed", "property_type"])["price"].median().rename("group_median_price")
    
    df = df.join(group_median, on=["neighbourhood_cleansed", "property_type"])
    
    # (price - median) / median price 
    # If its below 0 it's underpriced and overpriced over 0
    df["price_competitiveness"] = (
        (df["price"] - df["group_median_price"]) / df["group_median_price"]
    )
    
    # Normalised to 0–100% scale for easier interpretation 
    # The higher the more price competitive
    min_val = df["price_competitiveness"].min()
    max_val = df["price_competitiveness"].max()
    df["price_competitiveness"] = ((df["price_competitiveness"] - min_val) / (max_val - min_val) ) *100

    df["price_competitiveness"] = df["price_competitiveness"].round(2)

    df = df.rename(columns={"price_competitiveness": "price_competitiveness (100%)"})
    
    return df

def impute_price_column(df):
    # Flags rows where price or revenue was originally missing
    df["was_price_imputed"] = (
        df["price"].isna() | df["estimated_revenue_l365d"].isna()
    )

    # Impute price by neighbourhood + property_type median
    df["price"] = df.groupby(["neighbourhood_cleansed", "property_type"]
    )["price"].transform(lambda x: x.fillna(x.median()))

    return df

# If a listing has 0 reviews, all review-related values should be 0.
def fix_review_columns(df):
    
    review_cols = [
        "review_scores_rating","review_scores_cleanliness",
        "review_scores_value","reviews_per_month"]
    
    unreviewed_prop = df["number_of_reviews"] == 0
    
    for col in review_cols:
        df.loc[unreviewed_prop, col] = 0

    return df

# Predicts how likely a listing is to be booked based on demand signals.
def add_occupancy_potential(df):
    # Calculates the occupancy score components
    availability_inv = 1 - (df["availability_365"] / 365)
    rev_norm = df["estimated_revenue_l365d"] / df["estimated_revenue_l365d"].max()
    reviews_norm = df["reviews_per_month"] / df["reviews_per_month"].max()
    min_night_penalty = 1 / (df["minimum_nights"] + 1)

    # Weighted composite score
    score = (
        0.35 * availability_inv +
        0.30 * reviews_norm +
        0.25 * rev_norm +
        0.10 * min_night_penalty
    )

    # Normalise 0–1
    score = score / score.max()

    # Only add the final score column
    df["occupancy_potential"] = score.round(2)

    return df

def impute_minimum_beds(df):
    # Rename column once (safe if already renamed)
    if "beds" in df.columns:
        df = df.rename(columns={"beds": "minimum_beds"})

    # Flag: only rows where minimum_beds was missing
    df["was_beds_imputed"] = df["minimum_beds"].isna()

    # Fill missing minimum_beds using bedrooms
    df["minimum_beds"] = df.apply(
        lambda row: row["bedrooms"] if (pd.isna(row["minimum_beds"]) and not pd.isna(row["bedrooms"]))
        else row["minimum_beds"],
        axis=1
    )

    # Enforce logical rule: minimum_beds ≥ bedrooms
    df["minimum_beds"] = df[["minimum_beds", "bedrooms"]].max(axis=1)

    return df


def impute_bathrooms(df):
    # Flag only missing bathroom values
    df["was_bathrooms_imputed"] = df["bathrooms"].isna()

    # Helper function
    def estimate_bathrooms(bedrooms):
        if pd.isna(bedrooms):
            return None
        if bedrooms <= 2:
            return 1
        if bedrooms == 3:
            return 2
        # general guideline: 2 bathrooms per 3 bedrooms
        return math.ceil((bedrooms / 3) * 2)

    df["bathrooms"] = df.apply(
        lambda row: estimate_bathrooms(row["bedrooms"]) 
                    if pd.isna(row["bathrooms"]) 
                    else row["bathrooms"],
        axis=1
    )

    return df

def clean_airbnb_dataframe(df):
    # These just do the normal cleaning actions first
    df = filter_columns(df)
    df = convert_objects_to_string(df)
    df = convert_price_to_int(df)
    df = convert_bed_columns_to_int(df)
    df = convert_rates_to_int(df)
    df = convert_host_since(df)
    df = convert_superhost_to_bool(df)

    # Most of these impute a lot of critical values
    df = impute_price_column(df)
    df = fix_review_columns(df)
    df = impute_minimum_beds(df)
    df = impute_bathrooms(df)

    # These drop rows with excessive missigness
    df = drop_rows_with_missing_threshold(df, threshold=0.22)

    # This cleans the amenities values
    df = clean_amenities_column(df)

    # Feature engineering
    df = add_price_competitiveness(df)
    df = add_occupancy_potential(df)

    return df

filtered_df = clean_airbnb_dataframe(df)
filtered_df.dtypes



/var/folders/yw/8qjcs1g93ws0j0rgq361rm3h0000gn/T/ipykernel_4809/2547634568.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[obj_cols] = df[obj_cols].astype("string")
/var/folders/yw/8qjcs1g93ws0j0rgq361rm3h0000gn/T/ipykernel_4809/2547634568.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["price"] = (
/var/folders/yw/8qjcs1g93ws0j0rgq361rm3h0000gn/T/ipykernel_4809/2547634568.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

,id,property_type,room_type,price,estimated_revenue_l365d,accommodates,minimum_beds,bedrooms,bathrooms,review_scores_rating,...,host_is_superhost,review_scores_cleanliness,review_scores_value,host_acceptance_rate,was_price_imputed,was_beds_imputed,was_bathrooms_imputed,group_median_price,price_competitiveness (100%),occupancy_potential
0,13913,Private room in rental unit,Private room,7000,6440.0,1,1,1,1.0,4.85,...,True,4.80,4.78,96,False,False,False,7300.0,0.01,0.17
1,15400,Entire rental unit,Entire home/apt,14900,1341.0,2,1,1,1.0,4.80,...,False,4.87,4.74,50,False,False,False,22000.0,0.01,0.37
2,17402,Entire rental unit,Entire home/apt,41100,0.0,6,3,3,2.0,4.77,...,True,4.72,4.61,88,False,False,False,23400.0,0.03,0.61
3,24328,Entire townhouse,Entire home/apt,38450,NaN,2,1,1,1.0,4.90,...,False,4.91,4.65,11,True,True,True,38450.0,0.01,NaN
4,36274,Entire condo,Entire home/apt,21000,12600.0,4,1,1,1.0,4.53,...,False,4.54,4.54,98,False,False,False,15600.0,0.02,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96866,1508894090797273412,Entire rental unit,Entire home/apt,29800,0.0,2,2,2,2.0,0.00,...,False,0.00,0.00,97,False,False,False,16000.0,0.03,0.03
96867,1508900042872179492,Entire rental unit,Entire home/apt,6600,1188.0,2,1,1,1.0,5.00,...,False,5.00,5.00,100,False,False,False,16700.0,0.01,0.16
96868,1508926597927944565,Entire rental unit,Entire home/apt,35000,0.0,4,1,1,1.0,0.00,...,False,0.00,0.00,<NA>,False,False,False,12600.0,0.04,0.10
96869,1508962439633147670,Private room in rental unit,Private room,4000,0.0,1,1,1,1.0,0.00,...,False,0.00,0.00,<NA>,False,False,False,6200.0,0.01,0.13
